In [ ]:
!pip install python-resize-image
!pip install kaggle-cli

In [ ]:
# !pip install --no-cache-dir -I pillow
# !pip install Pillow==4.0.0
# !pip install PIL
# !pip install image

https://forums.fast.ai/t/attributeerror-module-pil-image-has-no-attribute-register-extensions/10689/5

In [ ]:
import os
import re
import urllib
import urllib.request
import urllib.parse
from PIL import Image
from resizeimage import resizeimage

In [ ]:
# !pip install Pillow
import PIL
from PIL import Image

In [ ]:
if not os.path.exists("dogsCats/train"):
    !kg download -u yourkagglelogin -p yourkagglepassword -c dogs-vs-cats
    !mkdir dogsCats;cd dogsCats;ls
    !mkdir dogsCats/test1
    !rm -rf test1.zip
    !mv train.zip dogsCats/
    !cd dogsCats; unzip train.zip
    !cd dogsCats;rm -rf test1.zip train.zip;ls

In [ ]:
if not os.path.exists("dogsCats/train/Dog") and not os.path.exists("dogsCats/train/Cat"):
    trainImgs = os.listdir('dogsCats/train/')
    !mkdir dogsCats/train/Dog dogsCats/train/Cat
    for img in trainImgs:
        if 'cat' in img:
            os.rename('dogsCats/train/' + img, 'dogsCats/train/Cat/' + img)
        elif 'dog' in img:
            os.rename('dogsCats/train/' + img, 'dogsCats/train/Dog/' + img)    

In [ ]:
# nbTrain = len(os.listdir('"dogsCats/train/Dog"')) + len(os.listdir('"dogsCats/train/Cat"'))
dogImgs = os.listdir('dogsCats/train/Dog')
catImgs = os.listdir('dogsCats/train/Cat')
if not os.path.exists("dogsCats/test1/Dog") and not os.path.exists("dogsCats/test1/Cat"):
    !mkdir dogsCats/test1/Dog dogsCats/test1/Cat
    nbDog = len(dogImgs)
    nbCat = len(catImgs)
    lim = nbDog * 0.2
    for ind, (dog, cat) in enumerate(zip(dogImgs, catImgs)):
        if ind > lim:
            print (ind)
            break;
        os.rename('dogsCats/train/Dog/' + dog, 'dogsCats/test1/Dog/' + dog)
        os.rename('dogsCats/train/Cat/' + cat, 'dogsCats/test1/Cat/' + cat)

In [ ]:
dogImgs = os.listdir('dogsCats/train/Dog')
catImgs = os.listdir('dogsCats/train/Cat')
nb_train_samples = len(dogImgs) + len(catImgs)
dogImgs = os.listdir('dogsCats/test1/Dog')
catImgs = os.listdir('dogsCats/test1/Cat')
nb_validation_samples = len(dogImgs) + len(catImgs)

In [ ]:
from skimage.io import imread, imsave
from skimage.transform import resize

In [ ]:
import tensorflow as tf
import keras as K

In [ ]:
print (tf.__version__, K.__version__)

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import cv2 as cv
%pylab inline

import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg

import tensorflow as tf
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU
# import keras_utils

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device = device_lib.list_local_devices()
#     print (local_device)
    return [x.name for x in local_device if x.device_type == 'GPU']

get_available_gpus()

In [ ]:
BATCH_SIZE = 64
INIT_LR = 5e-3 

In [ ]:
!cd dogsCats/ ; ls ; find train -type f | wc -l ; find test -type f | wc -l

In [ ]:
path = './dogsCats/train'
imgs = []

In [ ]:
for path, subdirs, files in os.walk(path):
    for name in files:
        imgs.append(os.path.join(path, name))

In [ ]:
print (imgs[:5])

In [ ]:
lbls = []
for img in imgs:
    if 'cat' in img:
        lbls.append(0)
    else:
        lbls.append(1)

In [ ]:
print (len(imgs), len(lbls))

In [ ]:
i = 0
count = 0
nbImgs = len(imgs)

In [ ]:
def reduce_dataset(cut, nbImgs, lbls, imgs):
    dog = cat = int((cut/2)*nbImgs)
    print (dog, cat)
    if int((cut*nbImgs)) != len(imgs):
        for ind, (img, lbl) in enumerate(zip(imgs, lbls)):
            if lbl == 0 and cat != 0:
                del lbls[ind]
                del imgs[ind]
                cat -= 1
            if lbl == 1 and dog != 0:
                del lbls[ind]
                del imgs[ind]
    nbImgs = len(imgs)
    print (nbImgs)

In [ ]:
imgs = []
path = 'dogsCats/train/'
for path, subdirs, files in os.walk(path):
    for name in files:
        imgs.append(os.path.join(path, name))

In [ ]:
cols = 8
rows = 3
#pour allouer un cadre
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(imgs))
        #on place chacune des images sur le cadre
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(Image.open(imgs[random_index]))
        ax.set_title('dog' if 'train/Dog' in imgs[random_index] else 'cat')
plt.show()

In [ ]:
def plot_history(history):
    # historique pour l'accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # historique pour la loss function
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def resnet():
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    pred = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=pred)
    return model

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

def vgg():
    base_model = VGG16(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    pred = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=pred)
    return model

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [ ]:
K.clear_session()
# model = resnet()
model = vgg()
# model = create_model()

In [ ]:
model.compile(optimizer = 'Adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
batch_size = 32
train_data_dir = 'dogsCats/train/'
validation_data_dir = 'dogsCats/test1/'
img_height, img_width = 150, 150
epochs = 10
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

In [ ]:
plot_history(history)

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

<b>Objectif:</b>
- atteindre les 90% sans overfitting